In [2]:
import sys
import os
if not any(path.endswith('textbook') for path in sys.path):
    sys.path.append(os.path.abspath('../../..'))
from textbook_utils import *

(sec:sql_transforming)=
# Transforming and Common Table Expressions

In this section, we show how to call functions to transform columns of data using built-in SQL functions. We also demonstrate how to use common table expressions to build up complex queries from simpler ones. As usual, we start by loading the database.

In [3]:
# Set up connection to database
import sqlalchemy
db = sqlalchemy.create_engine('sqlite:///babynames.db')

## SQL Functions

SQLite provides a variety of scalar functions, or functions that transform single data
values. When called on a column of data, SQLite will apply these functions on
each value in the column. In contrast, aggregation functions like `SUM` and
`COUNT` take a column of values as input and compute a single value as output.

SQLite provides a comprehensive list of the built-in scalar functions in [its online documentation][funcs]. For instance, to find the number of characters in each name, we use the
`LENGTH` function.

[funcs]: https://www.sqlite.org/lang_corefunc.html

In [4]:
query = ''' 
SELECT Name, LENGTH(Name)
FROM baby
LIMIT 10;
'''

pd.read_sql(query, db)

,Name,LENGTH(Name)
0,Liam,4
1,Noah,4
2,Oliver,6
...,...,...
7,Lucas,5
8,Henry,5
9,Alexander,9


Notice that the `LENGTH` function is applied to each value within the `Name`
column. 

:::{note}

Like aggregation functions, each implementation of SQL provides a different set
of scalar functions. SQLite has a relatively minimal set of functions, while
[PostgreSQL has many more][pg_funcs]. That said, almost all SQL implementations provide some
equivalent to SQLite's `LENGTH`, `ROUND`, `SUBSTR`, and `LIKE` functions. 

:::

[pg_funcs]: https://www.postgresql.org/docs/9.2/functions.html

Although scalar functions use the same syntax as an aggregation function, they behave differently. This
can result in confusing output if the two are mixed together in a single query:

In [12]:
query = ''' 
SELECT Name, LENGTH(Name), AVG(Count)
FROM baby
LIMIT 10;
'''

pd.read_sql(query, db)

,Name,LENGTH(Name),AVG(Count)
0,Liam,4,174.47


Here, the `AVG(Name)` computes the average of the entire `Count` column, but the output is confusing---a reader could easily think the average is related to the name `Liam`. For this reason, we must be careful when when scalar and aggregation functions
appear together within a `SELECT` statement. 

To extract the first letter of each name, we can use the `SUBSTR` function
(short for "substring"). As described in the documentation, the `SUBSTR`
function takes three arguments. The first is the input string, the second is
the position to begin the substring (1-indexed), and the third is the
length of the substring.

In [11]:
query = ''' 
SELECT Name, SUBSTR(Name, 1, 1)
FROM baby
LIMIT 10;
'''

pd.read_sql(query, db)

,Name,"SUBSTR(Name, 1, 1)"
0,Liam,L
1,Noah,N
2,Oliver,O
...,...,...
7,Lucas,L
8,Henry,H
9,Alexander,A


We can use the `AS` keyword to rename the column:

In [12]:
query = ''' 
SELECT *, SUBSTR(Name, 1, 1) AS Firsts
FROM baby
LIMIT 10;
'''

pd.read_sql(query, db)

,Name,Sex,Count,Year,Firsts
0,Liam,M,19659,2020,L
1,Noah,M,18252,2020,N
2,Oliver,M,14147,2020,O
...,...,...,...,...,...
7,Lucas,M,11281,2020,L
8,Henry,M,10705,2020,H
9,Alexander,M,10151,2020,A


After calculating the first letter of each name, our analysis aims to understand the popularity of first letters over time. To do this, we want to take the output of this SQL query and use it as a single step within a longer chain of operations.

SQL provides several options to break queries into smaller steps, which is helpful in more complex analyses like this one. The most common options for doing this are to create a new relation using a `CREATE TABLE` statement, create a new view using `CREATE VIEW`, or create a temporary relation using `WITH`. Each of these methods have different use cases. For simplicity, we only describe the `WITH` statement in this section and suggest readers to look over the SQLite documentation for details.

## Multistep Queries Using a `WITH` Clause

The `WITH` clause lets us assign a name to any `SELECT` query. Then, we can
treat that query as though it exists as a relation in the database just for the duration of the query.
SQLite calls these temporary relations *Common Table Expressions*.
For
instance, we can take the query above that calculates the first letter of each
name and call it `letters`:

In [14]:
query = ''' 
-- Create a temporary relation called letters by calculating
-- the first letter for each name in baby
WITH letters AS (
  SELECT *, SUBSTR(Name, 1, 1) AS Firsts
  FROM baby
)
-- Then, select the first ten rows from letters
SELECT *
FROM letters
LIMIT 10;
'''

pd.read_sql(query, db)

,Name,Sex,Count,Year,Firsts
0,Liam,M,19659,2020,L
1,Noah,M,18252,2020,N
2,Oliver,M,14147,2020,O
...,...,...,...,...,...
7,Lucas,M,11281,2020,L
8,Henry,M,10705,2020,H
9,Alexander,M,10151,2020,A


`WITH` statements are very useful since they can be chained together. We can
create multiple temporary relations in a `WITH` statement that each perform a
bit of work on the previous result, which lets us gradually build complicated
queries a step at a time. 

## Example: Popularity of "L" Names

We can use `WITH` statements to look at the popularity of names that start with the letter "L" over time. We'll group the temporary `letters` relation by the
first letter and year, then aggregate the `Count` column using a sum, then filter to get only names with the letter "L":

In [15]:
query = ''' 
WITH letters AS (
  SELECT *, SUBSTR(Name, 1, 1) AS Firsts
  FROM baby
)
SELECT Firsts, Year, SUM(Count) AS Count
FROM letters
WHERE Firsts = "L"
GROUP BY Firsts, Year;
'''

letter_counts = pd.read_sql(query, db)
letter_counts

,Firsts,Year,Count
0,L,1880,12799
1,L,1881,12770
2,L,1882,14923
...,...,...,...
138,L,2018,246251
139,L,2019,249315
140,L,2020,239760


This relation contains the same data as the one from {numref}`Chapter %s <ch:pandas>`. In that section, we make a plot of the `Count` column over time, which we omit here for brevity.

In this section, we introduced data transformations.
To transform values in a relation, we commonly use SQL functions like
`LENGTH()` or `SUBSTR()`.
We also explained how to build up complex queries using the `WITH` clause.